In [ ]:
library(tidyr)
library(brms)
library(dplyr)
library(lme4)
library(MASS)
library(kableExtra)
library(tidyverse)
library(broom)

In [2]:
data <- read.csv("data_ml/SharedResponses_combined_two_genders.csv")

In [3]:
data <- distinct(data)
data <- data %>% drop_na()
drops <- c("ResponseID", "ExtendedSessionID", "UserID", "Review_political")
data <- data[ , !(names(data) %in% drops)]
data$Saved <- factor(data$Saved)
data$Review_political_cat <- factor(data$Review_political_cat)

In [4]:
priors <- c(
  prior(normal(0, 1), class = "Intercept"), # Prior for the global intercept
  prior(student_t(1, 0, 2.5), class = "sd") # Prior for standard deviations of random effects
)

# Define the model
model_hierarchical <- brm(
  formula = Saved ~ 1 + (1 | Review_political_cat), # Model formula
  data = data,
  family = bernoulli("logit"), # Assuming 'Saved' is binary; using logistic regression
  prior = priors,
  chains = 4,
  iter = 2000,
  seed = 123
)

# Summary of the model
summary(model_hierarchical)

Compiling Stan program...

Start sampling




SAMPLING FOR MODEL 'anon_model' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.000183 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 1.83 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 2.789 seconds (Warm-up)
Chain 1:                1.772 seconds (Sampling)
Chain 1:                4.5

Warning message:
"There were 12 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them."
Warning message:
"Examine the pairs() plot to diagnose sampling problems
"
Warning message:
"Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#tail-ess"
Warning message:
"There were 12 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup"


 Family: bernoulli 
  Links: mu = logit 
Formula: Saved ~ 1 + (1 | Review_political_cat) 
   Data: data (Number of observations: 2094) 
  Draws: 4 chains, each with iter = 2000; warmup = 1000; thin = 1;
         total post-warmup draws = 4000

Multilevel Hyperparameters:
~Review_political_cat (Number of levels: 5) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     0.12      0.12     0.01     0.44 1.00      707     1340

Regression Coefficients:
          Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept    -0.16      0.08    -0.33     0.02 1.01      540      234

Draws were sampled using sampling(NUTS). For each parameter, Bulk_ESS
and Tail_ESS are effective sample size measures, and Rhat is the potential
scale reduction factor on split chains (at convergence, Rhat = 1).

In [5]:
model_non_hierarchical_with <- brm(
  formula = Saved ~ 1 + Review_political_cat,  # Intercept and Review_political_cat as fixed effect
  data = data,
  family = bernoulli("logit"),
  prior = c(
    prior(normal(0, 1), class = "Intercept"),
    prior(normal(0, 2.5), class = "b")  # Assuming a normal prior for fixed effects
  ),
  chains = 4,
  iter = 2000,
  seed = 123
)

Compiling Stan program...

Start sampling




SAMPLING FOR MODEL 'anon_model' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.000284 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 2.84 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 2.515 seconds (Warm-up)
Chain 1:                2.646 seconds (Sampling)
Chain 1:                5.1

In [10]:
library(loo)

# Hierarchical model
loo_hierarchical <- loo(model_hierarchical)

# Non-hierarchical models
loo_non_hierarchical_with <- loo(model_non_hierarchical_with)

# Compare models
loo_compare(loo_non_hierarchical_with, loo_hierarchical)

,elpd_diff,se_diff,elpd_loo,se_elpd_loo,p_loo,se_p_loo,looic,se_looic
model_hierarchical,0.000000,0.000000,-1446.203,3.784092,2.939388,0.04206903,2892.406,7.568183
model_non_hierarchical_with,-1.141091,1.428504,-1447.344,4.304561,5.108855,0.14227439,2894.688,8.609122
